In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import json
import time

In [7]:
def wait(seconds):
    time.sleep(seconds)

def append_to_json_file(data, file_path="results.json"):
    try:
        with open(file_path, "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        existing_data = []

    existing_data.append(data)

    with open(file_path, "w") as json_file:
        json.dump(existing_data, json_file, indent=4)

In [15]:
def fill_field(driver, field_id, value):
    field = driver.find_element(By.ID, field_id)
    field.send_keys(value)

def submit_form(driver):
    register_button = driver.find_element(By.ID, "register_submit")
    register_button.click()

In [16]:
def check_for_error(driver, error_id):
    try:
        error_element = driver.find_element(By.ID, error_id)
        return error_element.text if error_element.is_displayed() else ""
    except NoSuchElementException:
        return ""

In [17]:
def check_for_popup_error(driver):
    try:
        popup = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, "swal2-html-container"))
        )
        return popup.text
    except TimeoutException:
        return ""

In [18]:
def fill_input_fields(driver, first_name, last_name, emp_id, company_code):
    fill_field(driver, "first_name", first_name)
    fill_field(driver, "last_name", last_name)
    fill_field(driver, "emp_id", emp_id)
    fill_field(driver, "company_code", company_code)

In [21]:
def fill_form(test_case_title, first_name, last_name, emp_id, company_code):
    driver = webdriver.Chrome()
    error_text = ""

    try:
        driver.get("https://prog-demo-client.botnosticsolutions.ai/")  # Change this to the path where your form is located
        fill_input_fields(driver, first_name, last_name, emp_id, company_code)
        submit_form(driver)
        wait(5)

        error_text = check_for_error(driver, "error-email")
        if not error_text:
            error_text = check_for_error(driver, "error-pwd-format")
        if not error_text:
            error_text = check_for_error(driver, "error-pwd")

        popup_error_text = check_for_popup_error(driver)
        if popup_error_text:
            error_text = popup_error_text

        data = {
            "test_case_title": test_case_title,
            "first_name": first_name,
            "last_name": last_name,
            "emp_id": emp_id,
            "company_code": company_code,
            "error_text": error_text
        }

        append_to_json_file(data)

    except Exception as e:
        print("Error occurred during form filling:", e)
        import traceback
        traceback.print_exc()

        data = {
            "test_case_title": test_case_title,
            "first_name": first_name,
            "last_name": last_name,
            "emp_id": emp_id,
            "company_code": company_code,
            "error_text": error_text or "Exception occurred"
        }

        append_to_json_file(data)

    finally:
        wait(5)
        driver.quit()


In [23]:
# Test Cases
print("Running test cases...")

# Valid form submission
print("Test Case: Valid form submission")
fill_form("Valid form submission", "John", "Doe", "123", "Dem-1692959085")

Running test cases...
Test Case: Valid form submission


In [24]:
# Missing first name
print("Test Case: Missing first name")
fill_form("Missing first name", "", "Doe", "123", "Dem-1692959085")


Test Case: Missing first name


In [25]:
# Missing last name
print("Test Case: Missing last name")
fill_form("Missing last name", "John", "", "123", "Dem-1692959085")


Test Case: Missing last name


In [26]:
# Missing Employee ID
print("Test Case: Missing Employee ID")
fill_form("Missing Employee ID", "John", "Doe", "", "Dem-1692959085")

# Missing Company Code
print("Test Case: Missing Company Code")
fill_form("Missing Company Code", "John", "Doe", "123", "")

# Invalid Employee ID (non-numeric)
print("Test Case: Invalid Employee ID (non-numeric)")
fill_form("Invalid Employee ID (non-numeric)", "John", "Doe", "ABC", "Dem-1692959085")

# Invalid Company Code (missing 'Dem-')
print("Test Case: Invalid Company Code (missing 'Dem-')")
fill_form("Invalid Company Code (missing 'Dem-')", "John", "Doe", "123", "1692959085")

# Invalid Company Code (wrong format)
print("Test Case: Invalid Company Code (wrong format)")
fill_form("Invalid Company Code (wrong format)", "John", "Doe", "123", "Demo-1234567890")

Test Case: Missing Employee ID
Test Case: Missing Company Code
Test Case: Invalid Employee ID (non-numeric)
Test Case: Invalid Company Code (missing 'Dem-')
Test Case: Invalid Company Code (wrong format)
